In [1]:
import tensorflow as tf

CNN Prototype

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# Need to recalculate variance for the normalization layer.

model = Sequential([
    layers.Input(shape = (128,)),
    layers.Normalization(mean = 127, variance = 165000),
    layers.Dense(86, activation='relu'),
    layers.Dense(60, activation='relu'),
    layers.Dense(40, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(1, activation='relu')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 86)                11094     
_________________________________________________________________
dense_1 (Dense)              (None, 60)                5220      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2440      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 19,595
Trainable params: 19,595
Non-trainable params: 0
____________________________________________________

(this cell is just me experimenting with loading the data and training on it)

In [3]:
save_to_disk = False

TrainFeatures = pd.read_csv('trainFeatures.csv', header = 0, index_col = 0)
TrainTargets = pd.read_csv('trainTargets.csv', header = 0, index_col = 0)


model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mean_squared_error', 'mean_absolute_error'])
model.fit(TrainFeatures,TrainTargets,epochs = 500, batch_size = 4)
if save_to_disk:
    # Saving model to JSON and weights to H5.
    os.chdir('..')
    os.chdir('./models/')
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")
    print("Saved model to disk")
loss  = model.evaluate(ValFeatures, ValTargets)
print('Loss on Validation Set: ', loss)

NameError: name 'TrainFeatures' is not defined

RNN Prototype